# Collecting the datasets

In [ ]:
import cv2
import numpy
import time

In [ ]:
face_classifier=cv2.CascadeClassifier(r'C:\Users\dushy\Summer Training LW\Tasks\Task4\haarcascade_frontalface_default.xml')

def face_detector(img):
    gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces=face_classifier.detectMultiScale(gray,1.3,5)
    
    if faces is ():
        return None
    
    for (x,y,w,h) in faces:
        c_face=img[y:y+h,x:x+w]
        c_face=cv2.resize(c_face,(300,300))
    return c_face

cap=cv2.VideoCapture(0)
count=0
while True:
    ret,frame=cap.read()
    if face_detector(frame) is not None:
        face=face_detector(frame)
        count+=1
        g_face=cv2.cvtColor(face,cv2.COLOR_BGR2GRAY)
        
        #file_path='./faces_dataset/dushyanth/'+str(count)+'.jpg'
        file_path='./faces_dataset/sandhya/'+str(count)+'.jpg'
        cv2.imwrite(file_path,g_face)
        
        cv2.putText(g_face,str(count),(50,50),cv2.FONT_HERSHEY_COMPLEX,1,(0,255,0),2)
        cv2.imshow('Cropped face',g_face)
        time.sleep(0.1)
        
    else:
        print('Face not found')
        pass
    
    if cv2.waitKey(1)==13 or count==100:
        break
        
cap.release()
cv2.destroyAllWindows()
print("Samples collected successfully")

# Training the Model

In [ ]:
from os import listdir
from os.path import isfile,join

In [ ]:
#getting training data for training the model

#train_path ='./faces_dataset/dushyanth/'
train_path ='./faces_dataset/sandhya/'
file = [f for f in listdir(train_path) if isfile(join(train_path,f))]

#generating trainingdata and labels
training_data, labels = [], []

for i,files in enumerate(file):
    image_path=train_path+file[i]
    images=cv2.imread(image_path,cv2.IMREAD_GRAYSCALE)
    training_data.append(numpy.asarray(images,dtype=numpy.uint8))
    labels.append(i)
    
labels=numpy.asarray(labels,dtype=numpy.int32)

# creating and training the model

sandhya_model=cv2.face_LBPHFaceRecognizer.create()
sandhya_model.train(numpy.asarray(training_data),numpy.asarray(labels))
#dushyanth_model=cv2.face_LBPHFaceRecognizer.create()
#dushyanth_model.train(numpy.asarray(training_data),numpy.asarray(labels))
print("Model trained successfully")

# Running the Facial Recognition 

In [ ]:
import pywhatkit
import os
import subprocess
import json

In [ ]:
face_classifier=cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

def face_detect(img,size=0.5):
    gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces=face_classifier.detectMultiScale(gray,1.3,5)
    
    if faces is ():
        return img,[]
    
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,255),2)
        roi=img[y:y+h,x:x+w]
        roi=cv2.resize(roi,(300,300))
    return img,roi

def create_instance_ebs():
    instance = json.loads(subprocess.getoutput('aws ec2 run-instances --image-id ami-0ad704c126371a549 --count 1 --instance-type t2.micro --subnet-id subnet-d2957fb9 --security-group-ids sg-07a28161f98226946 --key-name myaws02'))
    print("Instance launched successfully")
    az = instance['Instances'][0]['Placement']['AvailabilityZone']
    insid = instance['Instances'][0]['InstanceId']
    ebs = json.loads(subprocess.getoutput('aws ec2 create-volume --volume-type gp2 --size 5 --availability-zone {0}'.format(az)))
    print("Volume created successfully")
    vid = ebs['VolumeId']
    print("Attaching...........")                                          
    attach = subprocess.getoutput('aws ec2 attach-volume --volume-id {0} --instance-id {1} --device /dev/xvdf'.format(vid,insid))
    return "Instance created and 5 GB EBS Volume Attached"

def send_whatsapp_message():
    pywhatkit.sendwhatmsg_instantly(phone_no="+91xxxxxxxxxx",message="Sandhya's face is recognized")
    pywhatkit.send_mail("source@gmail.com","xxxxxxx","Face Recoginition","Sandhya's face is recognized","receiver@yahoo.com")
    

cap=cv2.VideoCapture(0)

while True:
    ret,frame=cap.read()
    image,face=face_detect(frame)
    
    try:
        face=cv2.cvtColor(face,cv2.COLOR_BGR2GRAY)
        results=dushyanth_model.predict(face)
        results1=sandhya_model.predict(face)
        
        if results[1]<500:
            confidence=int(100*(1-(results[1])/400))
            user=1
            display_string=str(confidence)+'%confident, it is Dushyanth'  
        elif results1[1]<500:
            confidence=int(100*(1-(results[1])/400))
            user=2
            display_string=str(confidence)+'%confident, it is Sandhya'
            
        font=cv2.FONT_HERSHEY_COMPLEX
        cv2.putText(image,display_string,(100,120),font,1,(255,120,150),2)
        
        if confidence>90:
            cv2.putText(image,"Recognized",(250,160),font,1,(0,0,255),2)
            cv2.imshow('Face Recognition',image)
            if user==2:
                #send_whatsapp_message()
                print('Sandhya')
                break
            elif user==1:
                #print(create_instance_ebs())
                print('Dushyanth')
                break
                
        else:
            cv2.putText(image,"Not Recognized",(250,160),font,1,(0,0,255),2)
            cv2.imshow('Face Recognition',image)
            
    except:
            cv2.putText(image,"Cannot Detect Face",(200,160),cv2.FONT_HERSHEY_COMPLEX,1,(0,0,255),2)
            cv2.imshow('Face Recognition',image)
            pass
            
    if cv2.waitKey(1)==13:
            break
            
cap.release()
cv2.destroyAllWindows()